In [13]:
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')

import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from sklearn.metrics import log_loss
from IPython.display import display, HTML
import pickle

%matplotlib notebook



# Data
Here we have the initial pull of datasets include:
* game_df: this is game information for all games from 2008 to 2021 including unplayed games
* test_game_codes are going to be the game_codes of games in 2021 that are played (note: game_state_id 11 is finished game, 1 is Pre-Game)
* sim_game_codes are all games (played and unplayed) of 2021
* we split up all the game_codes into "played_game_codes" and "unplayed_game_codes"

#

In [14]:
game_df = pd.read_parquet(os.path.join(data_dir, 'game_data.parquet'))
game_df = game_df.sort_values(by='game_date', ascending=False).reset_index(drop=True)
game_codes = game_df.game_code.values
test_game_codes = game_df.loc[(game_df.season == 2021) & (game_df.game_state_id == 11), 'game_code'].values
sim_game_codes = game_df.loc[game_df.season == 2021, 'game_code'].values
played_game_codes = game_df.loc[game_df.game_state_id == 11, 'game_code'].values
unplayed_game_codes = game_df.loc[game_df.game_state_id != 11, 'game_code'].values
assert game_df.loc[~game_df.game_code.isin(unplayed_game_codes)].isna().sum().sum() == 0, 'Unexpected NaN values found in game data frame'
display(HTML(game_df.head(3).to_html(index=False)))
print('Games in sample: {0:d}'.format(game_codes.size))


game_code,game_date,home_team_id,away_team_id,season,home_team_abbrev,away_team_abbrev,week,home_score,away_score,status,game_state_id
2337702,2022-01-09 17:20:00,341,357,2021,LV,LAC,18,0,0,Pre-Game,1
2337697,2022-01-09 16:25:00,324,352,2021,Buf,NYJ,18,0,0,Pre-Game,1
2337688,2022-01-09 16:25:00,362,364,2021,TB,Car,18,0,0,Pre-Game,1


Games in sample: 3745


# Prior Data
Prior data is pulled from csv file and this will give us the inputs we need for the pre-game match predictions

In [15]:
prior_df = pd.read_csv(os.path.join(data_dir, 'game_priors.csv'))
display(HTML(prior_df.head(3).to_html(index=False)))

game_code,home_team_id,away_team_id,home_team_abbrev,away_team_abbrev,prior_home,prior_away,game_date
887191,329,347,Cle,Min,0.301116,0.696440,2009-09-13 13:00:00
887208,334,347,Det,Min,0.202565,0.796437,2009-09-20 13:00:00
887257,347,327,Min,Cin,0.676199,0.321909,2009-12-13 12:00:00


# Event Data
* Named event_df
* Is the play by play data from all games 2008 to 2021

# Input Features
Now that we have the datasets loaded we can load the features:
* prior_home: estimated probability of the home team winning at t=0
* prior_away: estimated probability of the away team winning at t=0
* home_team_has_ball: binary value for whether home team is in possession of the ball
* home_start_score: the score of the home team at the beginning of each play
* away_start_score: the score of the away team at the beginning of each play
* quarter: the current quarter/period the game is in (1-4 for all games, 5 if they are in the overtime period)
* play_start_time: numeric value of the time remaining in the quarter (900 at the beginning of the quarter, 0 at the end)
* yd_from_goal: the amount of yards between the line of scrimmage and the goal line for the team in possession of the ball
* down: the amount of downs that the team in possession of the ball has accumulate (1-4, down=-1 in plays that are not from scrimmage)
* ytg: the amount of yards between the current line of scrimmage and the first down line. (ytg=-1 in plays that are not from scrimmage)

# Target
remaining_exact_score: this is a numeric value for all the different combinations of remaining score (note: in this value sample, max_away_score=59 and max_home_score=62)
* for example if the current score is 17-24 (away_start_score=17 & home_start_score=24) and the final score is 27-30, then:
    *        remaining_exact_score = (27 - 17) + (62 + 1) * (30 - 24) = 422
* this ensures that all combinations of remaining exact scores are unique values


# Merged Table
full_df: the merged table of events_df and prior_df keeping only the input features and the target

In [16]:
events_df = pd.read_parquet(os.path.join(data_dir, 'event_data.parquet'))
max_away_score = np.max(game_df["away_score"])
max_home_score = np.max(game_df["home_score"])
input_names = ['prior_home', 'prior_away','home_team_has_ball', 'home_start_score', 'away_start_score', 'quarter', 'play_start_time', 'yd_from_goal', 'down', 'ytg']
output_name = 'remaining_exact_score'
events_df['remaining_exact_score'] = events_df["away_rest_of_game_score"] + \
                                     (max_away_score + 1) * events_df['home_rest_of_game_score']
full_df = events_df.merge(prior_df, on="game_code")
full_df = full_df[full_df[input_names+[output_name]].notna().all(axis=1)]


In [17]:
# Show how the data frame looks like at the beginning and end of a game
sample_game_code = full_df.sample(1).iloc[0]['game_code']
sample_game_info = prior_df.loc[prior_df.game_code == sample_game_code, ['home_team_abbrev', 'away_team_abbrev', 'game_date']].iloc[0].tolist()
print('\nData sample for game: {0} v {1} ({2})'.format(*sample_game_info))
display(HTML(full_df.loc[full_df.game_code == sample_game_code][["game_code", "nevent"] + input_names]
             .iloc[list(range(0, 5)) + list(range(-5, 0))]
             .to_html(index=False)))


Data sample for game: Ind v Min (2012-09-16 13:00:00)


game_code,nevent,prior_home,prior_away,home_team_has_ball,home_start_score,away_start_score,quarter,play_start_time,yd_from_goal,down,ytg
1204665,1,0.465865,0.531548,1,0,0,1,900.0,65,0,-1
1204665,2,0.465865,0.531548,0,0,0,1,900.0,105,0,-1
1204665,3,0.465865,0.531548,0,0,0,1,895.0,77,1,10
1204665,4,0.465865,0.531548,0,0,0,1,857.0,72,2,5
1204665,5,0.465865,0.531548,0,0,0,1,830.0,61,1,10
1204665,223,0.465865,0.531548,1,23,20,4,8.0,65,0,-1
1204665,224,0.465865,0.531548,0,23,20,4,8.0,109,0,-1
1204665,225,0.465865,0.531548,0,23,20,4,8.0,80,1,10
1204665,226,0.465865,0.531548,0,23,20,4,4.0,65,1,10
1204665,227,0.465865,0.531548,0,23,20,4,0.0,46,1,-1


# Train/Test Data Split
Training and test dataframes are created (2009-2020 are training seasons and 2021 is the test season)
# Model
This is the stored model that predicts the probability of each remaining score combination at each point of the game

In [18]:
test_game_codes = game_df.loc[(game_df.season == 2021) & (game_df.game_state_id == 11), 'game_code'].values
sim_game_codes = game_df.loc[game_df.season == 2021, 'game_code'].values
mask_test = full_df.game_code.isin(test_game_codes)

X_train = full_df.loc[~mask_test, input_names]
y_train = full_df.loc[~mask_test, output_name]
X_test = full_df.loc[mask_test, input_names].values
y_test = full_df.loc[mask_test, output_name].values
n_categories = (max_home_score + 1) * (max_away_score + 1)


clf = pickle.load(open(os.path.join(root_dir, "models/game_score.sav"), 'rb'))


# clf = MLPClassifier(
#     hidden_layer_sizes=[10,5],
#     activation='relu',
#     solver='adam',
#     alpha=0.0001,  # L2 regularization parameter
#     learning_rate_init=0.001,
#     batch_size=128,
#     random_state=1,
#     max_iter=50, #50
#     early_stopping=True,
#     validation_fraction=0.1,
#     n_iter_no_change=5,
#     verbose=True).fit(X_train, y_train)


In [19]:
def get_model_outputs(model, input_data, running_scores):
    raw_output = model.predict_proba(input_data)
    # Exact score outputs    
    score_probs = np.zeros((input_data.shape[0], n_categories))
    score_probs[:, clf.classes_] = raw_output
    # 1X2 prediction & team score outputs
    outcome_probs = np.zeros((input_data.shape[0], 3))
    home_score_probs = np.zeros((input_data.shape[0], max_home_score + 1))
    away_score_probs = np.zeros((input_data.shape[0], max_away_score + 1))
    for home_score in range(max_home_score + 1):
        ft_home_score = home_score + running_scores[:, 0]
        for away_score in range(max_away_score + 1):            
            ft_away_score = away_score + running_scores[:, 1]
            remaining_prob = score_probs[:, away_score + (max_away_score + 1) * home_score]
            # 1X2 - Home win
            outcome_probs[:, 0] = np.where(ft_home_score > ft_away_score,
                                           outcome_probs[:, 0] + remaining_prob,
                                           outcome_probs[:, 0])
            # 1X2 - Draw
            outcome_probs[:, 1] = np.where(ft_home_score == ft_away_score,
                                           outcome_probs[:, 1] + remaining_prob,
                                           outcome_probs[:, 1])
            # 1X2 - Away win
            outcome_probs[:, 2] = np.where(ft_home_score < ft_away_score,
                                           outcome_probs[:, 2] + remaining_prob,
                                           outcome_probs[:, 2])
            # Team scores
            home_score_probs[:, home_score] += remaining_prob
            away_score_probs[:, away_score] += remaining_prob
    return {
        'remaining_score': score_probs,
        'home_score': home_score_probs,
        'away_score': away_score_probs,
        'ft_outcome': outcome_probs
    }

## Usage example
Below is just a little bit of code that shows how the input and output data looks like for a few samples in a random game:

In [20]:
example_game_code = np.random.choice(test_game_codes)
example_indices = [0, 1, -2, -1]
example_input = full_df.sort_values("nevent").loc[full_df.game_code == example_game_code, input_names].values[example_indices]
full_df = full_df.sort_values(["game_code", "nevent"], ascending=True)
example_running_score = full_df.loc[full_df.game_code == example_game_code,
                                    ['home_start_score', 'away_start_score']].values[example_indices]
example_output = get_model_outputs(clf, example_input, example_running_score)
print('\nExample input data:')
display(HTML(pd.DataFrame(data=example_input, columns=input_names).to_html(index=False)))

# print('\nExample outputs (exact score):')
# print(example_output['remaining_score'][0][0:10])
# print(example_output['remaining_score'][0][10:20])
# print(example_output['remaining_score'][0][20:30])

print('\nExample outputs (home team score):')
display(HTML(pd.DataFrame(data=example_output['home_score'], columns=np.arange(max_home_score + 1)).to_html(index=False)))

print('\nExample outputs (away team score):')
display(HTML(pd.DataFrame(data=example_output['away_score'], columns=np.arange(max_away_score + 1)).to_html(index=False)))

print('\nExample outputs (1X2):')
display(HTML(pd.DataFrame(data=example_output['ft_outcome'], columns=['home win', 'draw', 'away win']).to_html(index=False)))


Example input data:


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability-tutorial/venv/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


prior_home,prior_away,home_team_has_ball,home_start_score,away_start_score,quarter,play_start_time,yd_from_goal,down,ytg
0.41689,0.580195,0.0,0.0,0.0,1.0,900.0,65.0,0.0,-1.0
0.41689,0.580195,1.0,0.0,0.0,1.0,900.0,93.0,0.0,-1.0
0.41689,0.580195,1.0,23.0,16.0,4.0,37.0,77.0,1.0,10.0
0.41689,0.580195,1.0,23.0,16.0,4.0,0.0,78.0,2.0,-1.0



Example outputs (home team score):


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
0.008766,0.000005,0.000277,0.015628,0.000007,0.000430,0.015225,0.022985,0.003161,0.011790,0.040492,0.004138,0.009875,0.039302,0.032033,0.009988,0.038337,0.068499,0.009525,0.022051,0.067578,0.035952,0.014152,5.137508e-02,0.065131,1.207232e-02,2.455323e-02,5.447441e-02,0.031123,1.382436e-02,3.397033e-02,4.834504e-02,8.493548e-03,1.561735e-02,3.440800e-02,2.060011e-02,7.041023e-03,1.514835e-02,2.541981e-02,3.596515e-03,7.013114e-03,1.076119e-02,7.880216e-03,4.833324e-03,4.572239e-03,9.321616e-03,1.456347e-03,2.239750e-03,4.063570e-03,4.242856e-03,3.500229e-04,1.980867e-03,1.824201e-03,4.800237e-04,4.310721e-04,1.521336e-03,5.327320e-04,1.095343e-04,2.198631e-04,5.263160e-04,4.574117e-11,0.0,2.507286e-04
0.007363,0.000002,0.000224,0.013520,0.000003,0.000328,0.013818,0.019601,0.002611,0.010624,0.036769,0.003368,0.009315,0.038499,0.028855,0.008557,0.037923,0.067863,0.008128,0.022264,0.071229,0.034376,0.013126,5.545154e-02,0.069076,1.089272e-02,2.600338e-02,5.897689e-02,0.030258,1.359092e-02,3.688894e-02,5.295192e-02,8.119504e-03,1.561857e-02,3.726366e-02,2.063440e-02,6.851421e-03,1.512347e-02,2.679028e-02,3.232505e-03,6.827795e-03,1.099350e-02,7.825904e-03,4.719385e-03,4.514002e-03,9.722298e-03,1.339014e-03,2.251428e-03,3.915158e-03,4.150545e-03,3.065681e-04,1.829080e-03,1.689979e-03,4.307016e-04,3.846859e-04,1.475202e-03,5.018139e-04,9.304476e-05,2.122066e-04,4.817784e-04,4.901733e-11,0.0,2.444303e-04
0.854347,0.000005,0.002811,0.073741,0.000004,0.000418,0.012118,0.044483,0.005117,0.000595,0.002617,0.000469,0.000033,0.000391,0.002487,0.000070,0.000025,0.000165,0.000005,0.000003,0.000009,0.000074,0.000005,6.770553e-07,0.000003,3.887596e-07,3.559861e-08,3.744735e-07,0.000003,1.977967e-07,1.045517e-07,1.382419e-07,2.636654e-07,5.858327e-11,7.177824e-09,1.205523e-08,2.620361e-08,1.004112e-07,1.882749e-09,1.291881e-09,2.971202e-09,1.321760e-09,2.495559e-09,3.633975e-15,1.537042e-16,2.079694e-12,1.323408e-10,1.082543e-16,2.477055e-11,1.744873e-09,2.857832e-16,1.126397e-14,4.455003e-16,3.766212e-16,2.021408e-27,8.909034e-18,1.196983e-16,8.688395e-12,2.585679e-30,1.763157e-16,1.171309e-14,0.0,7.482703e-11
0.875567,0.000006,0.002559,0.063952,0.000004,0.000335,0.010025,0.037867,0.004469,0.000441,0.001964,0.000362,0.000023,0.000282,0.001892,0.000051,0.000017,0.000114,0.000003,0.000002,0.000006,0.000051,0.000003,4.199992e-07,0.000002,2.563427e-07,2.161454e-08,2.270405e-07,0.000002,1.253339e-07,6.506564e-08,8.302622e-08,1.813072e-07,3.125641e-11,4.096438e-09,6.975359e-09,1.722933e-08,6.692021e-08,1.047412e-09,7.452895e-10,1.788268e-09,7.429699e-10,1.537103e-09,1.715924e-15,7.917354e-17,1.080038e-12,7.641288e-11,5.507673e-17,1.347086e-11,1.044885e-09,1.499397e-16,6.122369e-15,2.290830e-16,1.944089e-16,5.491503e-28,3.745210e-18,6.197886e-17,5.637149e-12,6.222252e-31,9.189187e-17,7.342483e-15,0.0,5.146484e-11



Example outputs (away team score):


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0.016219,0.000010,0.000513,0.021031,0.000017,0.001305,0.018457,0.033865,0.002867,0.017203,0.056379,0.003610,0.012931,0.052640,0.045055,0.008891,0.038192,0.084749,0.010104,0.020790,0.067502,0.039031,0.012556,4.674181e-02,0.063436,0.012132,2.593891e-02,4.762779e-02,0.027640,9.612106e-03,2.404479e-02,3.747533e-02,8.082076e-03,1.384958e-02,2.839715e-02,1.425874e-02,6.348066e-03,1.242133e-02,1.725402e-02,2.577568e-03,4.078822e-03,1.236072e-02,6.026505e-03,1.814505e-03,2.718243e-03,3.942434e-03,6.467371e-04,6.359473e-04,2.125224e-03,1.655287e-03,1.366728e-04,1.383711e-03,3.497620e-04,7.710269e-07,5.334107e-08,9.961657e-05,2.048336e-05,0.0,0.0,2.510309e-04
0.016264,0.000005,0.000424,0.021921,0.000009,0.001138,0.018528,0.034260,0.002428,0.017255,0.059355,0.002982,0.012855,0.054852,0.044616,0.008081,0.038130,0.088527,0.008907,0.020211,0.070646,0.038103,0.011532,4.830176e-02,0.065300,0.010908,2.563177e-02,4.833456e-02,0.026554,8.707304e-03,2.374675e-02,3.710142e-02,7.364570e-03,1.295041e-02,2.831477e-02,1.328486e-02,5.837320e-03,1.202279e-02,1.682103e-02,2.330801e-03,3.895022e-03,1.170572e-02,5.611339e-03,1.623545e-03,2.442853e-03,3.574436e-03,5.669149e-04,5.504434e-04,1.956337e-03,1.531846e-03,1.189647e-04,1.265064e-03,3.000037e-04,3.725597e-07,2.220930e-08,8.400914e-05,1.553040e-05,0.0,0.0,2.181924e-04
0.854396,0.000001,0.002606,0.057773,0.000001,0.000144,0.011706,0.056615,0.008202,0.000801,0.002154,0.000836,0.000230,0.000656,0.003104,0.000420,0.000022,0.000164,0.000021,0.000011,0.000013,0.000104,0.000009,5.836672e-07,0.000003,0.000004,4.266314e-08,1.160919e-07,0.000002,2.408137e-07,2.081153e-07,1.301006e-07,1.815256e-07,6.687847e-10,5.143770e-09,1.697153e-07,9.822441e-11,2.164044e-09,9.758156e-09,3.591824e-12,9.769840e-12,2.150405e-14,8.346889e-11,2.658836e-12,1.287009e-14,4.596360e-09,2.919883e-16,2.349275e-16,5.759658e-16,3.341431e-16,1.206211e-16,9.902754e-17,1.206325e-19,5.813151e-16,7.481534e-11,7.698772e-08,1.217208e-20,0.0,0.0,2.962264e-20
0.873945,0.000002,0.002380,0.050826,0.000001,0.000118,0.009721,0.049262,0.007181,0.000612,0.001637,0.000671,0.000177,0.000485,0.002405,0.000329,0.000015,0.000116,0.000015,0.000008,0.000008,0.000074,0.000006,3.620835e-07,0.000002,0.000003,2.553412e-08,6.943700e-08,0.000001,1.514169e-07,1.374011e-07,7.902041e-08,1.192584e-07,3.760802e-10,2.950317e-09,1.115723e-07,5.419897e-11,1.283993e-09,5.811477e-09,1.898830e-12,5.115476e-12,1.010579e-14,4.699304e-11,1.463181e-12,7.880559e-15,2.779529e-09,1.501500e-16,1.201213e-16,2.994445e-16,1.733183e-16,6.246547e-17,5.095752e-17,4.900351e-20,3.030903e-16,5.145750e-11,5.987052e-08,4.705408e-21,0.0,0.0,1.174410e-20



Example outputs (1X2):


home win,draw,away win
0.558969,0.027565,0.413465
0.580547,0.026754,0.392699
0.945993,0.042044,0.011964
0.952378,0.037609,0.010013


### Visualization of predictions
This is a simple interactive dashboard that lets the user select any game from the test data set and plots match outcome (1X2) and team score predictions. Just pick a game from the drop-down menu and click the "Plot" button.

In [21]:
game_info_df = game_df[['game_code', 'game_date', 'home_team_id', 'away_team_id', 'season', 'home_team_abbrev', 'away_team_abbrev']]
game_info_df['game_description'] = ['{0} {1} v {2} ({3})'.format(i.game_date, i.home_team_abbrev, i.away_team_abbrev, i.game_code) for _, i in game_info_df.iterrows()]
game_info_df = game_info_df.loc[game_info_df.game_code.isin(test_game_codes), ['game_code', 'game_description']]
game_info_df['ft_score'] = str(game_df.set_index('game_code').loc[game_info_df.game_code.values, 'away_score'].values) + "-" + str(game_df.set_index('game_code').loc[game_info_df.game_code.values, 'home_score'].values)

h_f = None
h_ax = None
h_ax_twin = None

def update_dashboard(change):
    global info_textbox
    info_textbox.value = ''
    if h_ax is not None:
        h_ax.cla()
        h_ax_twin.cla()
    plot()
    
def print_to_textbox(string, textbox_handle, clear_textbox=False):
    if textbox_handle is None:
        print(string)
    else:
        if clear_textbox or (textbox_handle.value == ''):
            textbox_handle.value = string
        else:
            textbox_handle.value += '<br>' + string

def plot():
    global h_f, h_ax, h_ax_twin
    if h_f is None:
        h_f, h_ax = plt.subplots(1, figsize=(9, 4))
        
    plot_game_code = game_info_df.set_index('game_description').loc[match_picker.value, 'game_code']
    plot_time = full_df.loc[full_df.game_code == plot_game_code, ['quarter', 'play_start_time']]
    plot_time = ((900 - plot_time.play_start_time) + \
                 (plot_time.quarter - 1 ) * 900).values
                 
    plot_input = full_df.loc[full_df.game_code == plot_game_code, input_names].values
    plot_running_score = full_df.loc[full_df.game_code == plot_game_code, ['home_start_score', 'away_start_score']].values
    plot_output = get_model_outputs(clf, plot_input, plot_running_score)    
    print_to_textbox('{0:d} prediction samples found'.format(plot_input.shape[0]), info_textbox)
    
    # Main axis (1X2 prediction)
    plot_x = np.append(np.vstack((plot_time[:-1], plot_time[1:])).flatten(order='F'), plot_time[-1])
    plot_y1 = np.vstack((plot_output['ft_outcome'][:, 0], plot_output['ft_outcome'][:, 0])).flatten(order='F')[:-1]
    plot_y2 = np.vstack((plot_output['ft_outcome'][:, 1], plot_output['ft_outcome'][:, 1])).flatten(order='F')[:-1]
    plot_y3 = np.vstack((plot_output['ft_outcome'][:, 2], plot_output['ft_outcome'][:, 2])).flatten(order='F')[:-1]
    h_ax.stackplot(plot_x, plot_y1, plot_y2, plot_y3,
                   labels=['home','draw','away'],
                   colors=['khaki', 'lightgray', 'lightskyblue'],
                   zorder=0)
    h_ax.legend(loc='upper left')
    if np.max(full_df.loc[full_df.game_code == plot_game_code, "quarter"].values) ==5:
        x_tick_pos = (60 * np.arange(0, 61, 15)).tolist()
        x_tick_str = ["Q1", "Q2", "Q3", "Q4", "OT"]
    else:
        x_tick_pos = (60 * np.arange(0, 60, 15)).tolist()
        x_tick_str = ["Q1", "Q2", "Q3", "Q4"]
        
    h_ax.set_xticks(x_tick_pos)
    h_ax.set_xticklabels(x_tick_str)
    h_ax.set_xlabel('Match time')
    h_ax.set_xlim(0, plot_x.max())
    h_ax.set_ylim(0, 1)
    h_ax.set_yticks(np.arange(0, 1.01, 0.25))
    h_ax.set_yticklabels(['{0:.0f}%'.format(100 * i) for i in np.arange(0, 1.01, 0.25)])    
    h_ax.set_ylabel('Probability')
    h_ax.set_title('{0}'.format(*game_info_df.set_index('game_code').loc[plot_game_code, ['game_description']].tolist()))
    
    # Twin axis (score prediction)
    if h_ax_twin is None:
        h_ax_twin = h_ax.twinx()
    # Home
    plot_home_score = plot_running_score[:, 0] + \
        np.sum(plot_output['home_score'] * np.tile(np.arange(max_home_score + 1), (plot_input.shape[0], 1)), axis=1)
    plot_home_score = np.vstack((plot_home_score, plot_home_score)).flatten(order='F')[:-1]
    h_ax_twin.plot(plot_x, plot_home_score, '-k', linewidth=1, zorder=10, color='darkgoldenrod', label='home score')
    # Away
    plot_away_score = plot_running_score[:, 1] + \
        np.sum(plot_output['away_score'] * np.tile(np.arange(max_away_score + 1), (plot_input.shape[0], 1)), axis=1)
    plot_away_score = np.vstack((plot_away_score, plot_away_score)).flatten(order='F')[:-1]
    h_ax_twin.plot(plot_x, plot_away_score, '-k', linewidth=1, zorder=10, color='dodgerblue', label='away score')
    
    max_score = int(np.ceil(np.append(plot_home_score, plot_away_score).max()))
    h_ax_twin.set_ylim(0, max_score)
    h_ax_twin.set_yticks(np.arange(0, max_score + 0.5))
    h_ax_twin.set_ylabel('Predicted score')
    
    h_f.tight_layout()
    
match_picker = widgets.Dropdown(
    options=game_info_df.game_description.values,
    description='Match'
)
run_btn = widgets.Button(
    description='Plot'
)
info_textbox = widgets.HTML(value="")

display(widgets.VBox([
    widgets.HBox([match_picker, run_btn]),
    info_textbox
]))
run_btn.on_click(update_dashboard)

/var/folders/dq/tcxv3s4j7y585_4jhfbdvrx00000gn/T/ipykernel_2617/3832173540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_info_df['game_description'] = ['{0} {1} v {2} ({3})'.format(i.game_date, i.home_team_abbrev, i.away_team_abbrev, i.game_code) for _, i in game_info_df.iterrows()]


## Preparing predictions for 2021 season simulation
This is the final section of this notebook, and its purpose is to use the model we have trained to generate match predictions for all games in the 2021/22 season. These will be used in the next notebook to simulate the outcome of the competition.

Since the purpose of this tutorial is not to be run live while games play, we can make the following simplification: we will assume that all 2021 games are either finished or not yet started, and will use observed results and predictions for them, resepectively. Therefore, we will gather input features and generate pre-game predictions only for unplayed games.

First, let's get results for played games:

In [22]:
results_df = game_df.loc[game_df.season==2021, ["game_code", "home_team_id", "away_team_id", "home_score", "away_score"]]
results_df = results_df.set_index('game_code').loc[np.intersect1d(sim_game_codes, played_game_codes)].reset_index()
team_names = game_df.loc[game_df.season==2021, ["home_team_id", "home_team_abbrev"]]
division_data = pd.read_parquet(os.path.join(data_dir, "division_data.parquet"))
team_names =team_names.rename(columns={'home_team_id': 'id', 'home_team_abbrev': 'name'}).drop_duplicates()
current_division_data = division_data.loc[division_data.season==2021].rename(columns={'team_id': 'id'})
team_names= team_names.merge(current_division_data)
team_names = team_names.to_dict(orient='records')

Now let's deal with unplayed games: get input features and pass to the model to get predictions:

In [23]:
sim_df = game_df.set_index('game_code').loc[unplayed_game_codes, ['home_team_id', 'away_team_id']].reset_index()

# Add prior match outcome probabilities
sim_df = sim_df.merge(
    right=prior_df[['game_code'] + np.intersect1d(input_names, prior_df.columns).tolist()],
    how='left', on='game_code'
)
sim_df['prior_home'] = np.where(np.isnan(sim_df["prior_home"]), np.mean(full_df["prior_home"]), sim_df["prior_home"])
sim_df['prior_away'] = np.where(np.isnan(sim_df["prior_away"]), np.mean(full_df["prior_away"]), sim_df["prior_away"])
sim_df['home_team_has_ball'] = 0
sim_df['home_start_score'] = 0
sim_df['away_start_score'] = 0
sim_df['quarter'] = 1
sim_df['play_start_time'] = 900
sim_df['yd_from_goal'] = 70
sim_df['down'] = 0
sim_df['ytg'] = -1
sim_input = sim_df[input_names].values
sim_running_score = sim_df[['home_start_score', 'away_start_score']].values
sim_output = get_model_outputs(clf, sim_input, sim_running_score)


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability-tutorial/venv/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


Finally, put results and predictions together and store in a JSON file for the simulation notebook:

In [24]:

# Create predictions object
predictions = []
for ind_game, game_code in enumerate(sim_df.game_code.values):
    predictions.append({
        'game_code': int(game_code),
        'home_team_id': int(sim_df.iloc[ind_game]['home_team_id']),
        'away_team_id': int(sim_df.iloc[ind_game]['away_team_id']),
        'pred_exact_score': sim_output['remaining_score'][ind_game].tolist(),
        'pred_outcome': sim_output['ft_outcome'][ind_game].tolist(),
        'current_score': sim_running_score[ind_game].tolist()
    })
sim_data = {
    'teams': team_names,
    'results': results_df.to_dict(orient='records'),
    'predictions': predictions,
    'prediction_params': {'max_home_score': int(max_home_score), 'max_away_score': int(max_away_score)}
}
with open(os.path.join(data_dir, 'simulation_inputs.json'), 'w') as f:
    json.dump(sim_data, f)
os.system('say "done"')


0